In [1]:
import pandas as pd
import numpy as np
import zipfile
import os
import json
import time
import multiprocessing as mp
import matplotlib.pyplot as plt

In [2]:
# dir
dir = '../simple phs/phs_2019/'
filename = f'{dir}phs_2019_playoffs.csv'
df = pd.read_csv(filename)

In [3]:
df.head()

,start_time,match_id,stage,map_type,map_name,player,team,stat_name,hero,stat_amount
0,8/31/2019 1:09,30172,Overwatch League 2019 Post-Season,CONTROL,Ilios,Elsa,Chengdu Hunters,All Damage Done,All Heroes,19059.670270
1,8/31/2019 1:09,30172,Overwatch League 2019 Post-Season,CONTROL,Ilios,Elsa,Chengdu Hunters,Assists,All Heroes,22.000000
2,8/31/2019 1:09,30172,Overwatch League 2019 Post-Season,CONTROL,Ilios,Elsa,Chengdu Hunters,Average Time Alive,All Heroes,302.704223
3,8/31/2019 1:09,30172,Overwatch League 2019 Post-Season,CONTROL,Ilios,Elsa,Chengdu Hunters,Barrier Damage Done,All Heroes,8861.540116
4,8/31/2019 1:09,30172,Overwatch League 2019 Post-Season,CONTROL,Ilios,Elsa,Chengdu Hunters,Damage - Quick Melee,All Heroes,152.999995


In [27]:
dt = df.groupby(by = ['stat_name', 'team', 'player', 'hero']).mean()
dt.to_csv('2019_playoffs_stat_df.csv')